In [1]:
import pandas as pd
import numpy as np

import os
cwd = os.getcwd()
kaggle = cwd == "/kaggle/working"

# pretrain = pd.read_parquet("pretrain.parquet")
# train = pd.read_parquet("train.parquet")
# test = pd.read_parquet("test.parquet")

pretrain = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "pretrain.parquet")
train = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "train.parquet")
test = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "test.parquet")

#convert to string
train["text"] = train["text"].astype(str)
test["text"] = test["text"].astype(str)

#keep only the first 1000 rows
pretrain = pretrain[:1000]
train = train[:1000]
test = test[:1000]

In [2]:
import torch
from torch import nn
from transformers import TrainingArguments, Trainer, AutoTokenizer, XLMRobertaTokenizerFast, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup, TrainerCallback, TrainerControl, TrainingArguments
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
config = model.config
tokenizer.model_max_length = config.max_position_embeddings
# model.classifier.out_proj = nn.Sequential(
#     nn.Linear(config.hidden_size, 1),
#     nn.Sigmoid()
# )

# tokenizer = AutoTokenizer.from_pretrained("microsoft/mdeberta-v3-base")
# model = AutoModelForSequenceClassification.from_pretrained("microsoft/mdeberta-v3-base", num_labels=1, ignore_mismatched_sizes=True)




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116_nocublaslt.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.6\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116_nocublaslt.dll...


c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\ProgramData\Anaconda3 did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin'), WindowsPath('C:/Program Files/NVIDIA/CUDNN/v8.x'), WindowsPath('C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/libnvvp'), WindowsPath('C:/ProgramData/Anaconda3/Library/usr/bin'), WindowsPath('C:/Users/Timothe/.dotnet/tools'), WindowsPath('E:/texlive/2022/bin/win32')}
  warn(msg)
c:\ProgramData\Anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
Some weights of XLMRobertaF

In [3]:
# for param in model.parameters():
#     param.requires_grad = False
# for param in model.classifier.parameters():
#     param.requires_grad = True

# for param in model.roberta.encoder.layer[-1].parameters():
#     param.requires_grad = True
# for param in model.roberta.encoder.layer[-2].parameters():
#     param.requires_grad = True


from transformers.models.xlm_roberta.modeling_xlm_roberta import XLMRobertaLayer

# Get the configuration of one existing layer to use as a template
template_config = model.config

# Manually create new layers using the template configuration
new_layers = [XLMRobertaLayer(template_config) for _ in range(2)]

# Initialize the new layers with random weights
for layer in new_layers:
    layer.apply(model._init_weights)

# Append new layers to the existing stack of layers
model.roberta.encoder.layer.extend(new_layers)

# Update the config to reflect the new number of layers
model.config.num_hidden_layers += 2

# Freeze the original layers
for param in model.roberta.encoder.layer[:-2].parameters():
    param.requires_grad = False

# Check the architecture (should now have 14 layers in the encoder)
# print(model)


In [4]:
from torch.utils.data import Dataset
import torch
import numpy as np

max_length = 128

def encode_texts(tokenizer, texts):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return torch.tensor(input_ids), torch.tensor(attention_masks)

pretrain_x, pretrain_attention_mask = encode_texts(tokenizer, pretrain['text'])
pretrain_y = torch.tensor(np.array(pretrain['label'].tolist()), dtype=torch.float32)

train_x, train_attention_mask = encode_texts(tokenizer, train['text'])
train_y = torch.tensor(np.array(train['label'].tolist()), dtype=torch.float32)

test_x, test_attention_mask = encode_texts(tokenizer, test['text'])
test_y = torch.tensor(np.array(test['label'].tolist()), dtype=torch.float32)



class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, label):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.label = label

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'label': self.label[idx],
        }

pretrain_dataset = CustomDataset(pretrain_x, pretrain_attention_mask, pretrain_y)
train_dataset = CustomDataset(train_x, train_attention_mask, train_y)
val_dataset = CustomDataset(test_x, test_attention_mask, test_y)

print(train_x.shape, train_attention_mask.shape, train_y.shape, test_x.shape, test_attention_mask.shape, test_y.shape, pretrain_x.shape, pretrain_attention_mask.shape, pretrain_y.shape)


torch.Size([1000, 128]) torch.Size([1000, 128]) torch.Size([1000]) torch.Size([1000, 128]) torch.Size([1000, 128]) torch.Size([1000]) torch.Size([1000, 128]) torch.Size([1000, 128]) torch.Size([1000])


In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from datasets import load_metric

# Function to compute metrics
def compute_metrics(predictions, labels):
    metric = load_metric("accuracy")
    metric_f1 = load_metric("f1")
    accuracy = metric.compute(predictions=predictions.argmax(-1), references=labels)
    f1 = metric_f1.compute(predictions=predictions.argmax(-1), references=labels, average='macro')
    return {"accuracy": accuracy["accuracy"], "f1": f1}


# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=999)
model = model.cuda()

# Training function
def train(model, lr_per_epoch, train_dataset, val_dataset):
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4)

    
    # Training loop
    for epoch in range(len(lr_per_epoch)):
        model.train()
        lr = lr_per_epoch[epoch]
        optimizer.param_groups[0]['lr'] = lr
        for batch in train_loader:
            inputs = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['label'].cuda()

            outputs = model(inputs, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

        # Validation loop
        model.eval()
        all_predictions = []
        all_labels = []
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['input_ids'].cuda()
                attention_mask = batch['attention_mask'].cuda()
                labels = batch['label'].cuda()

                outputs = model(inputs, attention_mask=attention_mask)
                logits = outputs.logits
                all_predictions.append(logits.cpu())
                all_labels.append(labels.cpu())

        all_predictions = torch.cat(all_predictions)
        all_labels = torch.cat(all_labels)
        metrics = compute_metrics(all_predictions, all_labels)
        print(f"Epoch {epoch+1} - {metrics}")

# Train the model
train(model, [1e-5, 1e-5, 1e-5], pretrain_dataset, val_dataset)
train(model, [1e-5, 1e-5, 1e-5], train_dataset, val_dataset)

# Save the model
torch.save(model.state_dict(), '/kaggle/working/model.pt' if kaggle else 'model.pt')

c:\ProgramData\Anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\Timothe\AppData\Local\Temp/ipykernel_22496/2589682640.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


Epoch 1 - {'accuracy': 0.681, 'f1': {'f1': 0.405116002379536}}
Epoch 2 - {'accuracy': 0.681, 'f1': {'f1': 0.405116002379536}}


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:64                                                                                   │
│                                                                                                  │
│   61 │   │   print(f"Epoch {epoch+1} - {metrics}")                                               │
│   62                                                                                             │
│   63 # Train the model                                                                           │
│ ❱ 64 train(model, [1e-5, 1e-5, 1e-5], pretrain_dataset, val_dataset)                             │
│   65 train(model, [1e-5, 1e-5, 1e-5], train_dataset, val_dataset)                                │
│   66                                                                                             │
│   67 # Save the model                                                                            │
│                                                                                                  │
│ in train:32                                                                                      │
│                                                                                                  │
│   29 │   │   lr = lr_per_epoch[epoch]                                                            │
│   30 │   │   optimizer.param_groups[0]['lr'] = lr                                                │
│   31 │   │   for batch in train_loader:                                                          │
│ ❱ 32 │   │   │   inputs = batch['input_ids'].cuda()                                              │
│   33 │   │   │   attention_mask = batch['attention_mask'].cuda()                                 │
│   34 │   │   │   labels = batch['label'].cuda()                                                  │
│   35                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt